<a href="https://colab.research.google.com/github/ZaatarX/TESTEPYSPARK-Confitec/blob/main/TESTEPYSPARK_Confitec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste 1

## Instalando Spark

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845500 sha256=898df8f7ef76942500335f361cec064b94ba9f9bfcf46a0fbf06c12c154d5143
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [5]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [6]:
!pip install -q findspark

## OS Var

In [7]:
import os

In [8]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [9]:
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

## Iniciando Spark

In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Starting Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [12]:
spark

## Abrindo Arquivo de Parquet

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
confitec_test = spark.read.parquet(
    '/content/drive/MyDrive/confitec/parquet'
)

In [23]:
confitec_test.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)



In [47]:
confitec_test.show()

+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+--------------------+
|               Title|               Genre|         GenreLabels| Premiere|             Seasons|SeasonsParsed|EpisodesParsed|     Length|MinLength|MaxLength|    Status|Active|Table|Language|         dt_inclusao|
+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+--------------------+
|      House of Cards|     Political drama|     political,drama| 1-Feb-13|6 seasons, 73 epi...|            6|            73| 42–59 min.|       42|       59|     Ended|     0|Drama| English|2021-03-16T21:20:...|
|       Hemlock Grove|     Horror/thriller|     horror,thriller|19-Apr-13|3 seasons, 33 epi...|            3|            33| 45–58 min.|       45|       58|

## 1) String para Datetime

In [17]:
from pyspark.sql.types import StringType
from pyspark.sql import functions as f

In [85]:
confitec_fix_dt_inclusao = confitec_test\
                        .withColumn(
                            "dt_inclusao",
                            f.substring(confitec_test.dt_inclusao, 0, 10)
                        )

In [87]:
confitec_fix_dt_inclusao.show(5)

+--------------------+----------------+----------------+---------+--------------------+-------------+--------------+----------+---------+---------+-------+------+-----+--------+-----------+
|               Title|           Genre|     GenreLabels| Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|Table|Language|dt_inclusao|
+--------------------+----------------+----------------+---------+--------------------+-------------+--------------+----------+---------+---------+-------+------+-----+--------+-----------+
|      House of Cards| Political drama| political,drama| 1-Feb-13|6 seasons, 73 epi...|            6|            73|42–59 min.|       42|       59|  Ended|     0|Drama| English| 2021-03-16|
|       Hemlock Grove| Horror/thriller| horror,thriller|19-Apr-13|3 seasons, 33 epi...|            3|            33|45–58 min.|       45|       58|  Ended|     0|Drama| English| 2021-03-16|
|Orange Is the New...|    Comedy-drama|    comedy-

In [108]:
confitec_fixed_datetime = confitec_fix_dt_inclusao\
                .withColumn(
                    "Premiere",
                    f.to_date(confitec_fix_dt_inclusao.Premiere\
                              .cast(StringType()), 'd-MMM-yy')
                )\
                .withColumn(
                    "dt_inclusao",
                    f.to_date(confitec_fix_dt_inclusao.dt_inclusao\
                              .cast(StringType()), 'yyyy-MM-d')
                )

In [109]:
confitec_fixed_datetime.show(5)

+--------------------+----------------+----------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+-----+--------+-----------+
|               Title|           Genre|     GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|Table|Language|dt_inclusao|
+--------------------+----------------+----------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+-----+--------+-----------+
|      House of Cards| Political drama| political,drama|2013-02-01|6 seasons, 73 epi...|            6|            73|42–59 min.|       42|       59|  Ended|     0|Drama| English| 2021-03-16|
|       Hemlock Grove| Horror/thriller| horror,thriller|2013-04-19|3 seasons, 33 epi...|            3|            33|45–58 min.|       45|       58|  Ended|     0|Drama| English| 2021-03-16|
|Orange Is the New...|    Comedy-drama|    co

## 2) Ordernar dados

In [114]:
confitec_ordered = confitec_fixed_datetime\
        .select("*")\
        .orderBy(['Active', 'Genre'], ascending=[False, True])

In [115]:
confitec_ordered.show()

+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+-----------+
|               Title|             Genre|       GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|          Table|Language|dt_inclusao|
+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+-----------+
|Knights of the Zo...|            Action|            action|2019-07-19|                 TBA|            0|             0|       TBA|        0|        0|Pending|     1|          Anime| English| 2021-03-16|
|       Kengan Ashura|            Action|            action|2019-07-31|                 TBA|            0|             0|       TBA|        0|        0|Pending|     1|          Ani

## 3) Remover Duplicados & trocar TBA

In [135]:
from pyspark.sql.functions import trim

In [139]:
confitec_trim = confitec_ordered\
            .withColumn('Title', trim('Title'))

In [140]:
confitec_distinct = confitec_trim.distinct()

In [141]:
print(f'Distinct: {confitec_distinct.count()}')
print(f'Duplicates: {confitec_ordered.count()}')

Distinct: 358
Duplicates: 358


In [120]:
from pyspark.sql.functions import regexp_replace

In [121]:
confitec_seasons = confitec_distinct\
            .withColumn(
                'Seasons', regexp_replace('Seasons', 'TBA', 'a ser anunciado')
            )

In [125]:
confitec_seasons.show()

+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+----------+------+---------------+----------+-----------+
|               Title|               Genre|         GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength|    Status|Active|          Table|  Language|dt_inclusao|
+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+----------+------+---------------+----------+-----------+
|                0.03|     Science fiction|     science-fiction|2016-11-25|3 seasons, 26 epi...|            3|            26|38–51 min.|       38|       51|   Pending|     1|ForeignLanguage|Portuguese| 2021-03-16|
|      13 Reasons Why|  Teen drama/mystery|  teen,drama,mystery|2017-03-31|2 seasons, 26 epi...|            2|            26|49–70 min.|       4

## 4) Adicionar timestamp

In [126]:
from pyspark.sql.functions import current_timestamp

In [127]:
confitec_timestamp = confitec_seasons\
                .withColumn("Data de Alteração", current_timestamp())

In [128]:
confitec_timestamp.show(5)

+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+----------+------+---------------+----------+-----------+--------------------+
|               Title|             Genre|       GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength|    Status|Active|          Table|  Language|dt_inclusao|   Data de Alteração|
+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+----------+------+---------------+----------+-----------+--------------------+
|                0.03|   Science fiction|   science-fiction|2016-11-25|3 seasons, 26 epi...|            3|            26|38–51 min.|       38|       51|   Pending|     1|ForeignLanguage|Portuguese| 2021-03-16|2023-05-14 19:21:...|
|      13 Reasons Why|Teen drama/mystery|teen,drama,mystery|2017-03-31|2 sea

## 5) Traduzir colunas

In [129]:
confitec_timestamp.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: date (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: date (nullable = true)
 |-- Data de Alteração: timestamp (nullable = false)



In [131]:
confitec_ptbr = confitec_timestamp\
            .withColumnRenamed('Title', 'Título')\
            .withColumnRenamed('Genre', 'Gênero')\
            .withColumnRenamed('GenreLabels', 'EtiquetasDeGênero')\
            .withColumnRenamed('Premiere', 'Data de Lançamento')\
            .withColumnRenamed('Seasons', 'Temporadas')\
            .withColumnRenamed('SeasonsParsed', 'TemporadasAnalisadas')\
            .withColumnRenamed('EpisodesParsed', 'EpisódiosAnalisados')\
            .withColumnRenamed('Length', 'Duração')\
            .withColumnRenamed('MinLength', 'DuraçãoMínima')\
            .withColumnRenamed('MaxLength', 'DuraçãoMáxima')\
            .withColumnRenamed('Status', 'Condição')\
            .withColumnRenamed('Active', 'Ativo')\
            .withColumnRenamed('Table', 'Tabela')\
            .withColumnRenamed('Language', 'Língua')

In [132]:
confitec_ptbr.printSchema()

root
 |-- Título: string (nullable = true)
 |-- Gênero: string (nullable = true)
 |-- EtiquetasDeGênero: string (nullable = true)
 |-- Data de Lançamento: date (nullable = true)
 |-- Temporadas: string (nullable = true)
 |-- TemporadasAnalisadas: long (nullable = true)
 |-- EpisódiosAnalisados: long (nullable = true)
 |-- Duração: string (nullable = true)
 |-- DuraçãoMínima: long (nullable = true)
 |-- DuraçãoMáxima: long (nullable = true)
 |-- Condição: string (nullable = true)
 |-- Ativo: long (nullable = true)
 |-- Tabela: string (nullable = true)
 |-- Língua: string (nullable = true)
 |-- dt_inclusao: date (nullable = true)
 |-- Data de Alteração: timestamp (nullable = false)



## 6) Escrever um CSV

In [134]:
confitec_ptbr.coalesce(1).write.csv(
    path='/content/drive/MyDrive/confitec/csv',
    mode='overwrite',
    sep=';',
    header=True
)

## 7) Bucket da Amazon

In [ ]:
# como não tenho um bucket da amazon devo supor que seja algo assim
filepath = 'bucket_confitec/testes/csv'
confitec_ptbr.coalesce(1).write.format('csv').option('header','true',sep=';').mode("append").save("s3://{filepath}")

# Teste 2

In [142]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [143]:
import numpy as np

In [145]:
matrix_a = np.random.randint(0, 100, (4, 4))
matrix_b = np.random.randint(0, 100, (4, 4))

In [146]:
m_produto = np.matmul(matrix_a, matrix_b)

In [149]:
produto = [[0,0,0,0],
         [0,0,0,0],
         [0,0,0,0],
         [0,0,0,0]]

In [150]:
# sem numpy
for i in range(len(matrix_a)):
   for j in range(len(matrix_b[0])):
       for k in range(len(matrix_b)):
           produto[i][j] += matrix_a[i][k] * matrix_b[k][j]

In [151]:
print(f'Matriz A: {matrix_a}')
print(f'Matriz B: {matrix_b}')
print(f'Matriz Produto: {m_produto}')
print(f'Produto: {produto}')

Matriz A: [[23 86 43 16]
 [29  5 18  8]
 [18 89 16 31]
 [88 75 46 47]]
Matriz B: [[78 59 26 35]
 [93 73 90  5]
 [68 46 53 94]
 [73 15 10 39]]
Matriz Produto: [[13884  9853 10777  5901]
 [ 4535  3024  2238  3044]
 [13032  8760  9636  3788]
 [20398 13488 11946  9612]]
Produto: [[13884, 9853, 10777, 5901], [4535, 3024, 2238, 3044], [13032, 8760, 9636, 3788], [20398, 13488, 11946, 9612]]
